# Pre-Processing

In [3]:
%pip install pdfplumber
%pip install langchain
%pip install langchain_elasticsearch
%pip install langchain_huggingface
%pip install uuid
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import os

#Get the file path of the pdf in the current directory
def get_pdf_path():
    list_of_files = os.listdir("./data_source/train_source")
    return list_of_files

In [18]:
import re
import pdfplumber

def pdf2text(pdf_path):
    pdf = pdfplumber.open(pdf_path)
    pages = pdf.pages
    text = ""
    for page in pages:
        sub = page.extract_text()
        text += sub

    # 특수문자 제거
    cleaned_text = re.sub(r'[^\w\s.,]', '', text)

    # .를 기준으로 문장단위 chunk 생성
    tokenized_sentence = cleaned_text.split(".")

    return tokenized_sentence

In [19]:
from uuid import uuid4

pdf_path = get_pdf_path()

parsed_text = []

for path in pdf_path:
    parsed_text.append(pdf2text("./data_source/train_source/"+path))

In [23]:
from langchain_core.documents import Document

documents = []

for single_pdf in parsed_text:
    for sen in single_pdf:
        doc = Document(
            page_content=sen,
            metadata={"source": f"{pdf_path}"},
        )
        documents.append(doc)

uuids = [str(uuid4()) for _ in range(len(documents))]

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore

In [27]:
embeddings = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-mpnet-base-v2",
    # model_kwargs={'device':'cuda'}
)

client = ElasticsearchStore(
    es_url="https://agi.veritasgarage.com:443", 
    index_name="dacon_nori_index", 
    es_user="elastic", 
    es_password="sukbeom", 
    embedding=embeddings)

client.add_documents(documents=documents, ids=uuids)

['16ba3406-97af-4db2-8bbb-a94354acaa63',
 'd9d56d15-8dd5-47cd-8797-340325ba740c',
 '04de5bfd-9538-49dd-bd17-57348db2b2df',
 '8fa75948-8fcd-4df7-9a07-21567f09a24a',
 '012f8b88-951d-47a4-b978-e5ee85be3870',
 '01b7aa81-19e9-46a8-8b31-f89e1d5b7914',
 '3fb22421-7a23-48c6-aba5-8475a3222a0f',
 '7a697a18-8997-4bdd-8952-cdc2c972ffd0',
 '8f044b9d-3736-4c74-868f-aaaabe81acde',
 '1889266b-0157-45cb-b91b-1262175c1197',
 '29b1b210-e750-487a-b37b-7cee210a0f89',
 '9e01ab6a-1ca5-474b-a906-5e6d832776cf',
 '3745bf22-0c10-427d-89b9-8f9122a7773e',
 'c0c29c38-16fa-4306-9e82-901a71a7883c',
 '1cb09d03-04a3-42bc-8101-89c9995516db',
 'ec2906be-2ae7-4014-9afe-15b0014e2bac',
 '26cfb57a-fb04-49de-ac85-6752ab04500a',
 '0d363ad6-702d-4fd0-851a-966baf0bf297',
 '6f872961-c707-47d4-ab57-bfb5013d354c',
 '551b5eff-c4f0-4c85-b18f-80a6dfd1f3a6',
 '209a076e-786f-402d-b1e5-03b0fcd3b40e',
 'fcdab96a-8367-41b3-8ea7-7036f608407d',
 '200f5a38-a175-4464-8d5e-e8dfce218616',
 'e9ed2f0d-bec7-4b9a-b8c7-71563256f6da',
 '6da446d9-5936-